In [39]:
import csv
import os
import re
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
total_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'EDITORIAL', 'SUCURSAL', 'DESC_SUCURSAL', 'COD_BARRAS_LIBRO', 'NUM_TITULO', 'SIG_TOPOGRAFICA',
    'DEWEY', 'II_SUMARIO', 'I_SUMARIO', 'EDICION', 'LOCALIZACION_EJEMPLAR', 'DESC_LOCALIZACION_EJEMPLAR', 'POLITICA',
    'DESC_POLITICA_EJEMPLAR', 'ISBN', 'FECHA_REGISTRO_EJEMPLAR', 'FECHA_ADICION_EJEMPLAR', 'FECHA_PUB',
    'ESTADO_EJEMPLAR', 'PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION', 'NIVEL_EDUCACION', 'AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO', 'FECHA_REAL_DEVOLUCION',
]

In [3]:
used_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'SUCURSAL', 'COD_BARRAS_LIBRO', 'DEWEY', 'DESC_LOCALIZACION_EJEMPLAR',
    'FECHA_PUB','PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION','NIVEL_EDUCACION','AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO','SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO','FECHA_REAL_DEVOLUCION'
]

# Préstamos

In [4]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 18.8 s, sys: 654 ms, total: 19.5 s
Wall time: 19.4 s


In [5]:
prestamos_df.shape

(3054360, 20)

In [6]:
prestamos_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA,FECHA_PRESTAMO,FECHA_REAL_DEVOLUCION,DATE_PRESTAMO,DATE_DEVOLUCION
0,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429527,CTA,22/01/2015,04/02/2015,2015-01-22,2015-02-04
1,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88416381,CTA,31/03/2015,07/04/2015,2015-03-31,2015-04-07
2,Quisiera tener ...,"Zoboli, Giovanna.",ARM,2284458,853,Infantil,2010,mx,,,,ARM,ARM,ARM,88381353,CTA,07/11/2015,17/11/2015,2015-11-07,2015-11-17
3,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,Adm. de empresas,ARM,ARM,ARM,88007424,CEM,16/01/2015,16/01/2015,2015-01-16,2015-01-16
4,La fantástica leyenda de: la princesa y el dragón,"Alins, Sonia.",ARM,2288896,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429529,CTA,21/01/2015,28/01/2015,2015-01-21,2015-01-28


# Sucursales

In [7]:
sucursales = {
    'ARM': {'name': 'Armenia', 'latitude': 4.53, 'longitude': -75.68},
    'BAR': {'name': 'Barranquilla', 'latitude': 10.963889, 'longitude': -74.796389},
    'BLAA': {'name': 'Luis Angel Arango', 'latitude': 4.59677, 'longitude': -74.072848},
    'BUC': {'name': 'Bucaramanga', 'latitude': 7.133333, 'longitude': -73},
    'BUE': {'name': 'Buenaventura', 'latitude': 3.877222, 'longitude': -77.026667},
    'CAL': {'name': 'Cali', 'latitude': 3.420556, 'longitude': -76.522222},
    'CAR': {'name': 'Cartagena', 'latitude': 10.4, 'longitude': -75.5},
    'CASAGO': {'name': 'Casa Gómez Campuzano', 'latitude': 4.6628468, 'longitude': -74.0512116},
    'CATMAN': {'name': 'Catalogación y Mantenimiento de Colecciones', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'CUC': {'name': 'Cúcuta', 'latitude': 7.894167, 'longitude': -72.503889},
    'ESPE': {'name': 'Especial material audiovisual', 'latitude': 4.59677, 'longitude': -74.072848},
    'FLO': {'name': 'Florencia', 'latitude': 1.614167, 'longitude': -75.611667},
    'GIR': {'name': 'Girardot', 'latitude': 4.305, 'longitude': -74.801667},
    'HON': {'name': 'Honda', 'latitude': 5.204167, 'longitude': -74.741667},
    'IBA': {'name': 'Ibagué', 'latitude': 4.433333, 'longitude': -75.233333},
    'IEC': {'name': 'Investigaciones Económicas', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'IPI': {'name': 'Ipiales', 'latitude': 0.830278, 'longitude': -77.644444},
    'LET': {'name': 'Leticia', 'latitude': -4.205278, 'longitude': -69.932778},
    'MAN': {'name': 'Manizales', 'latitude': 5.1, 'longitude': -75.55},
    'MED': {'name': 'Medellín', 'latitude': 6.230833, 'longitude': -75.590556},
    'MON': {'name': 'Montería', 'latitude': 8.75, 'longitude': -75.883333},
    'NEI': {'name': 'Neiva', 'latitude': 2.998611, 'longitude': -75.304444},
    'PAS': {'name': 'Pasto', 'latitude': 1.207778, 'longitude': -77.277222},
    'PER': {'name': 'Pereira', 'latitude': 4.814278, 'longitude': -75.694558},
    'POP': {'name': 'Popayán', 'latitude': 2.454167, 'longitude': -76.609167},
    'QUI': {'name': 'Quibdó', 'latitude': 5.692222, 'longitude': -76.658056},
    'RIO': {'name': 'Riohacha', 'latitude': 11.544167, 'longitude': -72.906944},
    'SAN': {'name': 'San Andrés', 'latitude': 12.583333, 'longitude': -81.7},
    'SIN': {'name': 'Sincelejo', 'latitude': 9.295, 'longitude': -75.396111},
    'STM': {'name': 'Santa Marta', 'latitude': 11.241944, 'longitude': -74.205278},
    'TUN': {'name': 'Tunja', 'latitude': 5.533333, 'longitude': -73.366667},
    'VAL': {'name': 'Valledupar', 'latitude': 10.483333, 'longitude': -73.25},
    'VIL': {'name': 'Villavicencio', 'latitude': 4.15, 'longitude': -73.633333},
}

# Country codes

## LOC/MARC

http://www.loc.gov/marc/countries/countries_code.html

In [8]:
loc_marc_df = pd.read_csv('codes-loc-marc-countries.csv', sep='\t', names=['code','country'])
loc_marc_sr = pd.Series(data=list(loc_marc_df['country']), index=list(loc_marc_df['code']))

In [9]:
loc_marc_sr.head()

aa                          Albania
abc                         Alberta
-ac     Ashmore and Cartier Islands
aca    Australian Capital Territory
ae                          Algeria
dtype: object

## ISO

http://www.nationsonline.org/oneworld/country_code_list.htm

In [10]:
iso_df = pd.read_csv('codes-iso-countries.csv', sep='\t', names=['country', 'iso-2', 'iso-3', 'code'])
iso2_sr = pd.Series(data=list(iso_df['country']), index=map(lambda x: str(x).lower(), iso_df['iso-2']))
iso3_sr = pd.Series(data=list(iso_df['country']), index=map(lambda x: str(x).lower(), iso_df['iso-3']))

In [11]:
iso2_sr.head()

af       Afghanistan
ax     Aland Islands
al           Albania
dz           Algeria
as    American Samoa
dtype: object

In [12]:
iso3_sr.head()

afg       Afghanistan
ala     Aland Islands
alb           Albania
dza           Algeria
asm    American Samoa
dtype: object

## Mine

In [13]:
missing_codes_dict = {
    'uk': 'United Kingdom',
    'en': 'England',
    'spa': 'Spain',
    'CK': 'Colombia',
    'eng': 'England',
    'ger': 'Germany',
    'uy|': 'Uruguay',
    'SP': 'Spain',
    'AG': 'Argentina',
    'por': 'Portugal',
    'jap': 'Japan',
    'ur': 'Uruguay',
    'fl': 'Florida',
    'flo': 'Florida',
    'fre': 'French Guiana',
    'he': 'Heard Island and Mcdonald Islands',
    'ka': 'Kansas',
    'lou': 'Louisiana',
    'pu': 'Puerto Rico',
    'wa': 'Washington (State)',
}

# Autores

In [14]:
autores_df = pd.DataFrame()

## Nombre

In [15]:
prestamos_df['AUTOR'].value_counts()

                                                                409373
García Márquez, Gabriel, 1927-2014                               13207
Colombia.                                                        11526
Hergé, 1907-1983.                                                11510
Browne, Anthony, 1946-.                                           8653
Foucault, Michel, 1926-1984.                                      6849
Cortázar, Julio, 1914-1984.                                       6478
Nietzsche, Friedrich Wilhelm, 1844-1900.                          5876
Da Coll, Ivar, 1962-                                              5350
Kasza, Keiko.                                                     5336
Shakespeare, William, 1564-1616.                                  5012
Verne, Jules, 1828-1905.                                          4394
Instituto Colombiano de Normas Técnicas y Certificación.          4318
Dostoievski, Fedor Mijailovich, 1821-1881.                        4296
Rowlin

In [18]:
%%time

def process_autor(x):
    a = x['AUTOR'].strip(' ' + punctuation)
    a = a.replace(', ', ',')
    if a:
        b = re.findall('(.+),\d+', a)
        if b:
            a = b[0]
        c = a.split(',')
        if len(c) > 1:
            r = c[1] + ' ' + c[0]
        else:
            r = c[0]
    else:
        r = 'DESCONOCIDO'
    return r

autores_df['AUTOR'] = prestamos_df.apply(process_autor, axis=1)

CPU times: user 7min 26s, sys: 1.07 s, total: 7min 27s
Wall time: 7min 27s


In [20]:
autores_df['AUTOR'].value_counts()

DESCONOCIDO                                                     409373
Gabriel García Márquez                                           14397
Colombia                                                         11526
Hergé                                                            11510
Anthony Browne                                                    9653
Michel Foucault                                                   6849
Julio Cortázar                                                    6478
Friedrich Wilhelm Nietzsche                                       5876
Ivar Da Coll                                                      5350
Keiko Kasza                                                       5336
William Shakespeare                                               5012
José Saramago                                                     4818
Jules Verne                                                       4394
Instituto Colombiano de Normas Técnicas y Certificación           4333
Fedor 

## País ejemplar

In [17]:
total_countries = set(list(loc_marc_sr.values) + list(iso2_sr.values) + list(iso3_sr.values))

In [18]:
# %%time

# def process_pais(x):
#     a = x['PAIS_EJEMPLAR'].strip()
#     if a in loc_marc_sr:
#         r = loc_marc_sr[a]
#     elif '-' + a in loc_marc_sr:
#         r = loc_marc_sr['-' + a]
#     elif a in iso3_sr:
#         r = iso3_sr[a]
#     elif a in iso2_sr:
#         r = iso2_sr[a]
#     elif a in missing_codes_dict:
#         r = missing_codes_dict[a]
#     else:
#         r = 'DESCONOCIDO'
#     return r

# autores_df['PAIS_EJEMPLAR'] = prestamos_df.apply(process_pais, axis=1)

In [20]:
autores_df['AUTOR'].nunique()

145220

In [29]:
# c=0
# for x in autores_df['AUTOR'].unique():
#     a = x.replace(' ','').replace('-','').replace('.','').replace('\'','')
#     if not a.isalpha():
#         print(x)
#         c+=1
# c

In [19]:
for a in prestamos_df['AUTOR'].unique():
    b = re.findall('(.+),\d{2}', a)
    if b:
        print(a)

Congresos I y II de Filosofía (1996,1997 : Caldas)
Encuentros nacionales de filosofía (2000,2001 : Bogotá)
Suárez Ávila, Manuel,1968-


In [35]:
s=set()
for a in prestamos_df['AUTOR'].unique():
    b = re.findall('\w,\w', a)
    if b:
        s.add(a)

In [36]:
len(s)

64

In [37]:
s

{'Aldana Alzate,Gabriel, 1955-',
 'Angel,Daniel.',
 'Aragón Borja,Laura Edna.',
 'Arroyo Pizarro,Yolanda, 1970-',
 'Backer,Morton.',
 'Ballesteros Rodriguez,Jorge.',
 'Barlet Jaume,Xana.',
 'Bechara,Alvaro.',
 'Benavent García,José Manuel.',
 'Blumenthal,Ralph.',
 'Bonneto,n France.',
 'Bueno Chávez,Raúl, 1944-',
 'Burns,Roger George, 1937-',
 'Caballero,Martha.',
 'Calderón,Reinán, 1969-',
 'Cano Borrego,Pedro Damián, 1968-',
 'Cavazzoni,Ermanno, 1947-',
 'Chauvel,Pascal.',
 'Congresos I y II de Filosofía (1996,1997 : Caldas)',
 'Del Toma,Eugenio.',
 'Dorn,Nicholas.',
 'Díaz Covarrubias,Juan, 1837-1859.',
 'Encuentros nacionales de filosofía (2000,2001 : Bogotá)',
 'Fisac Badell,Taciana.',
 'García Gutiérrez,Emilio.',
 'García Rodríguez,Venancio.',
 'Gaster,Theodor Herzl, 1906-',
 'Handel,George Frideric, (1685-1759)',
 'Ibarra,Martín Gustavo.',
 'Jaeger,Paul, 1905-',
 'Jiménez Vargas,Liliana.',
 'Kahn,Peter B.',
 'Kermode,Frank, 1919-',
 'Lehninger,Albert L.,.',
 'Luchaire,M. Francoi

In [21]:
c=0
for x in autores_df['AUTOR'].unique():
    if ',' in x:
        print(x)
        c+=1
c

0

In [22]:
autores_df['AUTOR'].unique()

array(['Mon Daporta', 'Giovanna Zoboli', 'Lluís Borràs Perelló', ...,
       'Martha Maestre de Aroca', 'Cesar (Colombia). Gobernación',
       'Guzmán Torres Quintero'], dtype=object)

In [22]:
# autores_df['PAIS_EJEMPLAR'].value_counts()

In [32]:
autores_df['AUTOR'].value_counts().ix['Gabriel García Márquez']

14397

In [53]:
def f(x):
    a = x.strip(' ' + punctuation)
    a = a.replace(', ', ',')
    if a:
        b = re.findall('(.+),\d+', a)
        if b:
            a = b[0]
        c = a.split(',')
        if len(c) > 1:
            d = c[1] + ' ' + c[0]
        else:
            d = c[0]
#         d = d.strip(',')
#         d = d.replace(', ', ',')
#         if len(c) > 1:
#             d = c[1] + ' ' + c[0]
#         else:
#             d = c[0]
        r = d
    else:
        r = 'DESCONOCIDO'
    return r

In [56]:
prestamos_df['AUTOR'].unique()[:100]

array(['Daporta, Mon.', 'Zoboli, Giovanna.', 'Borràs Perelló, Lluís.',
       'Alins, Sonia.', '', 'Ayres, Charlie.', 'Wolfe, Gillian.',
       'Badstuber, Martina.', 'Aerssen, Ignacio van.',
       'Rius, Roser, 1965-', 'Fortes Torres, Antón, 1957-',
       'Cooper, Helen.', 'Pedrosa, Cyril.', 'Calvo, Mercedes.',
       'Bouchard, André.', 'Verplancke, Klaas.', 'Shone, Rob.',
       'Obiols, Anna.', 'Aliaga Sánchez, Roberto, 1976-',
       'Casalderrey, Fina, 1951-', 'Yeoman, John.', 'Le Huche, Magali.',
       'Collazos, Oscar, 1942-2015.', 'Waechter, Friedrich Karl.',
       'Gnone, Elisabetta, 1965-', 'Schwarz, Viviane.', 'Kim, Wan-doo.',
       'Gurrutxaga, Mikel.', 'Charlat, Benoit.', 'Suzy, Lee, 1974-',
       'Lee, Stan, 1922-.', 'Hergé, 1907-1983.',
       'Vallejo Betancur, Alejandro.', 'Nanclares, Silvia.',
       'Hohler, Franz, 1943-', 'Gutiérrez, Luz Myriam.',
       'Mejía Rivera, Orlando, 1961-', 'Cárdenas Ortiz, Alvaro Ramón.',
       'Johnson, Stephen L., 1949-', 'Mar

In [57]:
for a in prestamos_df['AUTOR'].unique()[:100]:
    print(f(a))

Mon Daporta
Giovanna Zoboli
Lluís Borràs Perelló
Sonia Alins
DESCONOCIDO
Charlie Ayres
Gillian Wolfe
Martina Badstuber
Ignacio van Aerssen
Roser Rius
Antón Fortes Torres
Helen Cooper
Cyril Pedrosa
Mercedes Calvo
André Bouchard
Klaas Verplancke
Rob Shone
Anna Obiols
Roberto Aliaga Sánchez
Fina Casalderrey
John Yeoman
Magali Le Huche
Oscar Collazos
Friedrich Karl Waechter
Elisabetta Gnone
Viviane Schwarz
Wan-doo Kim
Mikel Gurrutxaga
Benoit Charlat
Lee Suzy
Stan Lee
Hergé
Alejandro Vallejo Betancur
Silvia Nanclares
Franz Hohler
Luz Myriam Gutiérrez
Orlando Mejía Rivera
Alvaro Ramón Cárdenas Ortiz
Stephen L. Johnson
Isabel Minhós Martins
Charo Pita
Stephanie Rosenheim
Jimmy Liao
Quentin Blake
Jorge Luján
Tim Bowley
Yves Coppens
Alvaro Muñoz
Glenn Murphy
Rudyard Kipling
Benjamin Lacombe
Susana Henao Montoya
Isabel Pin
Nicholas Allan
Carla Baredes
Mariano Ribas
Pablo Pineau
Menena Cottin
Alvarito Cuevas
Fanuel Hanán Díaz
Gonzalo Moure Trenor
P.I Maltbie
Isabelle Carrier
Michel Van Zeveren
Ma

In [29]:
r = 'Orosio, Pablo, 390?-418?'.strip().strip('.')
s = re.findall('(.+), \d{3}', r)
print(r)
print(s)

Orosio, Pablo, 390?-418?
['Orosio, Pablo']


In [49]:
def get_pais_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            ?author wdt:P106 wd:Q36180 .
            ?author wdt:P19 ?pob .
            ?pob wdt:P17 ?country .
            ?country rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [57]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_pais_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            ?author rdf:type dbo:Writer .
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type dbo:Country .
            ?pob rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [156]:
def get_pais_autor_data(db, name, lang):
    if db == 'wdt':
        data = get_pais_autor_data_wkd(name, lang)
    elif db == 'dbp':
        data = get_pais_autor_data_dbp(name, lang)
    
    results = set(d['countryLabel']['value'] for d in data['results']['bindings'])
    if len(results) == 0:
        result = 'DESCONOCIDO'
    elif len(results) == 1:
        result = results.pop()
    else:
        print('MáS DE UN PAíS PARA', name, ':', results)
        result = results.pop()
    return result

In [135]:
def choose(opt_list):
    return Counter(opt_list).most_common(1)[0][0]

In [140]:
def get_pais_autor(name):
    r = 'DESCONOCIDO'
    opt_dict = {}
    for lang in ['en', 'es']:
        opt_dict[lang] = {}
        for db in ['dbp', 'wdt']:
            opt_dict[lang][db] = get_pais_autor_data(db, name, lang)
        opts = opt_dict[lang].values()
        if all(map(lambda x: x != 'DESCONOCIDO', opts)):
            r = choose(opts)
            break
    if r == 'DESCONOCIDO':
        opts = list(filter(lambda x: x != 'DESCONOCIDO', opt_dict['en'].values()))
        if opts:
            r = choose(opts)
    return r

In [141]:
get_pais_autor('Gabriel García Márquez')

'Colombia'

In [142]:
get_pais_autor('Juan Ramón Jiménez')

'Spain'

In [143]:
get_pais_autor('dsdfsdfsd')

'DESCONOCIDO'

In [ ]:
%%time

qqq=[]
for x in autores_df['AUTOR'].unique():
    qqq.append((x, get_pais_autor(x)))
ppp=pd.DataFrame(qqq, columns=['AUTOR', 'PAÍS'])
ppp.to_csv('autores_paises.csv', index=False)

MáS DE UN PAíS PARA Ludmila Zeman : {'Czechoslovakia', 'Czech Republic'}
MáS DE UN PAíS PARA Ludmila Zeman : {'República Checa', 'Checoslovaquia'}
MáS DE UN PAíS PARA David Shannon : {'United States of America', 'United Kingdom'}
MáS DE UN PAíS PARA David Shannon : {'Reino Unido', 'Estados Unidos'}
MáS DE UN PAíS PARA Mario Ramos : {'Belgium', 'Spain'}
MáS DE UN PAíS PARA Mario Ramos : {'España', 'Bélgica'}


In [152]:
autores_df['AUTOR'].nunique()

145132

In [155]:
qqq

[('Mon Daporta', 'DESCONOCIDO'),
 ('Giovanna Zoboli', 'DESCONOCIDO'),
 ('Lluís Borràs Perelló', 'DESCONOCIDO'),
 ('Sonia Alins', 'DESCONOCIDO'),
 ('DESCONOCIDO', 'DESCONOCIDO'),
 ('Charlie Ayres', 'DESCONOCIDO'),
 ('Gillian Wolfe', 'DESCONOCIDO'),
 ('Martina Badstuber', 'Germany'),
 ('Ignacio van Aerssen', 'DESCONOCIDO'),
 ('Roser Rius', 'DESCONOCIDO'),
 ('Antón Fortes Torres', 'DESCONOCIDO'),
 ('Helen Cooper', 'United Kingdom'),
 ('Cyril Pedrosa', 'DESCONOCIDO'),
 ('Mercedes Calvo', 'DESCONOCIDO'),
 ('André Bouchard', 'DESCONOCIDO'),
 ('Klaas Verplancke', 'DESCONOCIDO'),
 ('Rob Shone', 'DESCONOCIDO'),
 ('Anna Obiols', 'DESCONOCIDO'),
 ('Roberto Aliaga Sánchez', 'DESCONOCIDO'),
 ('Fina Casalderrey', 'Spain'),
 ('John Yeoman', 'DESCONOCIDO'),
 ('Magali Le Huche', 'DESCONOCIDO'),
 ('Oscar Collazos', 'Colombia'),
 ('Friedrich Karl Waechter', 'DESCONOCIDO'),
 ('Elisabetta Gnone', 'Italy'),
 ('Viviane Schwarz', 'DESCONOCIDO'),
 ('Wan-doo Kim', 'DESCONOCIDO'),
 ('Mikel Gurrutxaga', 'DESCONOC

In [147]:
aaa=pd.DataFrame([(1,2),(3,4),(5,6)], columns=['AUTOR', 'PAÍS'])

In [148]:
aaa

,AUTOR,PAÍS
0,1,2
1,3,4
2,5,6


In [ ]:
aaa.to_csv()